# Импортируем библиотеки и ключ openai из файла token

In [35]:
import os
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import warnings

token_file = open("token.txt", "r")
token = token_file.readline()
token_file.close()

os.environ["OPENAI_API_KEY"] = token

# Деление текста на фрагменты

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
coffee_zero  = """отличная кофемашина, бюджетная для семьи 2 человека, простая в использовании.
"1. Весьма удобно, после рожка - исчезли россыпи молотого кофе по всей кухне.

2. Имеется нормальный капучинатор, но я не пользовался еще, судя по отзывам качество посредственное, но его хотя бы не надо так тщательно вымывать после каждого использования, баклашку с молоком отцепили и поставили в холодильник, пениться будет лучше.

3. Несмотря на черный цвет - материалы не маркие и раз в неделю, можно просто стереть пыль даже сухой тряпкой.

4. Вместимости под зерна кофе хватает на дня 3-4, а лоток с водой и жмыхом примерно раз в сутки чищу."
Отлично готовит кофе.
Вкусный кофе
Круто варит кофе, лучше чем баристо!
автоматический капучинатор, большой выбор по настройке крепости и объему кофе, простота в обслуживании (легко моется стакан капучинатора, нет дурацкий шлангов, носиков и т.д)
Вкусный кофе. Старая машина была более шумная. Шикарная пенка. Все настроила под себя с легкостью. Места занимает немного на моей кухне.
Очень легкая в использовании и обслуживании. Во-первых легкоразборный и легкомоющийся капучинатор, во-вторых - собственный фильтр, гарантирующий минимальное образование накипи. Легко доливать воду,молоко. Легко мыть.
Вкусно готовит кофе на работе даже на поганой водопроводной воде (стоит фильтр).
Супер машинка, варит хороший кофе, вкусный горячий, все как надо
"- Быстро

- Просто

- Удобно"
Варит кофе отлично, быстро, легко моется
Надёжная,  лёгкая в освоении и эксплуатации
Встроенная кофемолка, прост в использовании
недорого
Вкусный кофе
Достойный внешний вид, сравнительно небольшие размеры, варит хорошо
Варит кофе вкусный кофе , фильтр для воды что идёт в комплекте видимо даёт эффект. В целом отличный аппарат пока что всё радует .
Красивая, простая в использовании, вкусно готовит КОФЕ!
удобно, кофе делает быстро,
Красивая , довольно тихая, простое обслуживание, неплохой кофе
Автоматический капучинатор, хорошая пенка.
"Хорошее качество сборки, производитель Румыния, не Китай.

Автоматический капучинатор.

Вкусный капучино, не плохой кофе.

Дизайн.

Легко моется и обслуживается.

Плотные таблетки из жмыха."
Варит хороший кофе. Капучинатор встроенный.
Отличная кофемашина.
Отличная кофемашина.
"Варит вкусный кофе.

Хорошая пенка.

Простая в использовании.

Удобно мыть.

Не требует постоянного обслуживания."
В принципе всего достаточно. Не сказал бы что шумит сильно.
-
Хороший кофе, когда разобрался легка в использовании, очень просто мыть.
Автоматический капучинатор - очень удобно!
"Вкусный кофе, различные варианты кофе. регулировка помола.  Использование молотого и зернового кофе.

"
Это первая домашняя кофемашина.
Надежная, удобная
компактная, капучинатор автоматический и легко снимается. регулировка высоты под разные кружки.
Ну однозначно, хороший аппарат для обычного пользователя
Отличная кофемашина
Отличная техника! Все устраивает и внешне и технически! Очень рады приобретением! Огромное спасибо!
Хороший кофе
Всё супер, о покупки несколько не пожалел.
Прост в использовании
"Полный автомат, включая капучино.

Простота очистки.

Стильный внешний вид."
Автоматический капучинатор
вкусный кофе
Исправно работает, варит вкусный кофе.
Все отлично! Готовит замечательный кофе.
Цена
Надежность, простота чистки и обслуживания.
Самый необходимый функционал для дома.Очень доволен..Есть небольшие нюансы как и с любой техникой. но на все дело привычки. В общем без нареканий
Вкусный кофе, красивый внешний вид
Качественная, хорошая кофемашина
Хорошо готовит латте
стабильная повторяемость качества результата. 3 главных вида кофе она умеет на отлично, и делает их довольно быстро. крышка зарнового бака защищена резиночкой, что не даст зернам терять аромат(так быстро). копрус выполнен из качественного и крепкого пластика, чтото там отломать будет нереально.
Пользуемся более месяца , результат превосходный , за это время использовано более 3 кг зернового кофе, капучино прекрасный , пена плотная и держится очень долго. Удобно настроили индивидуальный объем напитка, наслаждаемся ароматным кофе каждый день теперь.
Отличная кофемашина за свои деньги, кофе получается как надо. За свои деньги хороший вариант
-
Простота в использовании. Небольшие габариты. Варит вкусный кофе. Размер порции достаточный есть регулировка. Выходит кофе горячий
внешний вид

готовность кофе

легкость в использовании"
Отличная кофемашина
Очень вкусный кофе, можно регулировать и температуру (я не пью кипяток), и максимальное количество максимального количества (кружка 250, примерно 230 максимально) и таймер выключения. Для первой машины пушка бомба.
отличная кофемашина, бюджетная для семьи 2 человека, простая в использовании.
1. Весьма удобно, после рожка - исчезли россыпи молотого кофе по всей кухне.

2. Имеется нормальный капучинатор, но я не пользовался еще, судя по отзывам качество посредственное, но его хотя бы не надо так тщательно вымывать после каждого использования, баклашку с молоком отцепили и поставили в холодильник, пениться будет лучше.

3. Несмотря на черный цвет - материалы не маркие и раз в неделю, можно просто стереть пыль даже сухой тряпкой.

4. Вместимости под зерна кофе хватает на дня 3-4, а лоток с водой и жмыхом примерно раз в сутки чищу."
Отлично готовит кофе.
Вкусный кофе
Круто варит кофе, лучше чем баристо!
автоматический капучинатор, большой выбор по настройке крепости и объему кофе, простота в обслуживании (легко моется стакан капучинатора, нет дурацкий шлангов, носиков и т.д)
Вкусный кофе. Старая машина была более шумная. Шикарная пенка. Все настроила под себя с легкостью. Места занимает немного на моей кухне."""
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100,
#     chunk_overlap=0,
#     separators = ['. ', "\n"]
# )
# splitted_data = splitter.split_text(coffee_zero)
# splitted_data
path = ''
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="coffee_zero.txt", loader_cls=TextLoader,
                         loader_kwargs=text_loader_kwargs)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )"]
)
splitted_texts = splitter.split_documents(docs)
splitted_texts

[Document(page_content='отличная кофемашина, бюджетная для семьи 2 человека, простая в использовании.', metadata={'source': 'coffee_zero.txt'}),
 Document(page_content='"1. Весьма удобно, после рожка - исчезли россыпи молотого кофе по всей кухне.', metadata={'source': 'coffee_zero.txt'}),
 Document(page_content='\n2. Имеется нормальный капучинатор, но я не пользовался еще, судя по отзывам качество посредственное, но его хотя бы не надо так тщательно вымывать после каждого использования, баклашку с молоком отцепили и поставили в холодильник, пениться будет лучше.', metadata={'source': 'coffee_zero.txt'}),
 Document(page_content='\n3. Несмотря на черный цвет - материалы не маркие и раз в неделю, можно просто стереть пыль даже сухой тряпкой.', metadata={'source': 'coffee_zero.txt'}),
 Document(page_content='\n4. Вместимости под зерна кофе хватает на дня 3-4, а лоток с водой и жмыхом примерно раз в сутки чищу."', metadata={'source': 'coffee_zero.txt'}),
 Document(page_content='Отлично гото

# Сохраняем БД, а потом пробуем прочитать :)

In [37]:
directory = ''
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=splitted_texts,
    embedding=embedding,
    persist_directory=directory
)

print(vectordb._collection.count()) # Проверка
vectordb.persist() # Пишем

# Читаем
vectordb = Chroma(
    persist_directory=directory,
    embedding_function=embedding
)
print(vectordb._collection.count()) # Проверка

171
171


# Пишем ответ с помощью GPT

In [39]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
# os.environ["LANGCHAIN_API_KEY"] = ""
# os.environ["LANGCHAIN_PROJECT"] = "Game Zone"
warnings.filterwarnings("ignore")

question = "Расскажи кратко про эту кофемашину"
docs_k = 3

from pprint import pprint as pp

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": docs_k})#search_type = "mmr"

template = """Ты - полезный ИИ консультант для нашего магазина бытовой техники.
Используйте следующие фрагменты контекста (Context), чтобы ответить на вопрос в конце (Question).
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Сначала убедитесь, что прикрепленный текст имеет отношение к вопросу.
Если текст не имеет отношения к вопросу, просто скажите, что текст не имеет отношения.
Используйте максимум 10 предложений. Дайте ответ как можно более понятным, рассказывая кратко про все особенности.
Context: {context}
Question: {question}
Helpful Answer:""" 
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

#langchain.debug = True
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens = 300)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    #chain_type="refine",
    #chain_type="map_reduce",
)
result = qa_chain({"query": question})
pp(result)
langchain.debug = False


{'query': 'Расскажи кратко про эту кофемашину',
 'result': 'Эта кофемашина является отличным выбором для приготовления '
           'вкусного кофе. Она обладает всеми необходимыми функциями для '
           'приготовления различных видов кофе, включая эспрессо, капуччино и '
           'латте. Кофемашина имеет удобный и интуитивно понятный интерфейс, '
           'что делает ее использование простым и удобным. Она также обладает '
           'стильным дизайном, который отлично впишется в интерьер вашей '
           'кухни. Кофемашина имеет высокую производительность и быстро '
           'готовит кофе, что позволяет вам наслаждаться свежим напитком в '
           'любое время. Кроме того, она легко чистится и обслуживается, что '
           'делает ее уход простым и удобным. В целом, эта кофемашина '
           'предлагает отличное сочетание качества, функциональности и '
           'удобства использования',
 'source_documents': [Document(page_content='Отличная кофемашина.\n"Варит вкус